In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


2025-05-24 23:23:44.085720: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-24 23:23:44.100145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748100224.117279   45973 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748100224.122436   45973 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748100224.135932   45973 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [32]:
dataset_df = pd.read_csv("data/cvd_v2.csv", sep=',')
train, test = train_test_split(dataset_df, test_size=0.1)
print(len(train), 'train examples')
print(len(test), 'test examples')


61816 train examples
6869 test examples


In [33]:
data_features_train = train.copy()
data_labels_train = data_features_train.pop('cardio')
data_features_train = np.array(data_features_train)

data_features_test = test.copy()
data_labels_test = data_features_test.pop('cardio')
data_features_test = np.array(data_features_test)



In [4]:
normalize = layers.Normalization()
print(normalize)
normalize.adapt(data_features_train)

def build_model():
    model = tf.keras.Sequential([
        normalize,
        layers.Dense(256, activation='relu'),
        layers.Dropout(rate=0.5),
        layers.Dense(256, activation='relu',kernel_regularizer='l1'),
        layers.Dropout(rate=0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    optimizer = tf.keras.optimizers.Nadam()
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

<Normalization name=normalization, built=False>


I0000 00:00:1748100228.571004   45973 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38134 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:8b:00.0, compute capability: 8.0


In [35]:
deep_ensemble = []
for ind in range(3):
    model = build_model()
    model.fit(data_features_train, data_labels_train, epochs=60, batch_size=32, verbose=1)
    deep_ensemble.append(model)
    model.save(f'keras_model_hdf5_version_{ind}.h5')

Epoch 1/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.6755 - loss: 4.1916
Epoch 2/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7260 - loss: 0.7079
Epoch 3/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7253 - loss: 0.6972
Epoch 4/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7242 - loss: 0.6973
Epoch 5/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7246 - loss: 0.6908
Epoch 6/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7225 - loss: 0.6908
Epoch 7/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7244 - loss: 0.6872
Epoch 8/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7281 - loss: 0.6784
Epoch 9/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7227 - loss: 0.6836
Epoch 10/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7248 - loss: 0.6803
Epoch 11/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7240 - loss: 0.6823
Epoch 12/60
1932/1932 ━━━━━━━━

Epoch 1/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.6617 - loss: 4.2057
Epoch 2/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7209 - loss: 0.7123
Epoch 3/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7251 - loss: 0.6991
Epoch 4/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7228 - loss: 0.6920
Epoch 5/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7256 - loss: 0.6918
Epoch 6/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7219 - loss: 0.6892
Epoch 7/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7243 - loss: 0.6890
Epoch 8/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7247 - loss: 0.6855
Epoch 9/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7234 - loss: 0.6841
Epoch 10/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7180 - loss: 0.6860
Epoch 11/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7256 - loss: 0.6812
Epoch 12/60
1932/1932 ━━━━━━━━

Epoch 1/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.6775 - loss: 4.1814
Epoch 2/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7220 - loss: 0.7131
Epoch 3/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7236 - loss: 0.7000
Epoch 4/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7227 - loss: 0.6920
Epoch 5/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7220 - loss: 0.6882
Epoch 6/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7222 - loss: 0.6863
Epoch 7/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7230 - loss: 0.6886
Epoch 8/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7212 - loss: 0.6906
Epoch 9/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7227 - loss: 0.6839
Epoch 10/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7248 - loss: 0.6818
Epoch 11/60
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7217 - loss: 0.6835
Epoch 12/60
1932/1932 ━━━━━━━━

In [ ]:
predictions = []

for model in deep_ensemble:

    y_pred = model.predict(data_features_test)
    # print(y_pred)
    predictions.append(y_pred)
# 进行投票集成
ensemble_prob = np.mean(predictions, axis=0)
ensemble_pred = (ensemble_prob >= 0.5).astype(int)
# print(ensemble_pred.flatten())
# print(data_labels_test.values)
# 计算准确率
accuracy = np.mean(ensemble_pred.flatten() == data_labels_test.values)
print("Ensemble Accuracy:", accuracy)



174/215 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step

2025-05-25 00:23:58.161894: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30', 188 bytes spill stores, 188 bytes spill loads

2025-05-25 00:23:58.263418: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30', 36 bytes spill stores, 36 bytes spill loads

2025-05-25 00:23:58.389617: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_30', 268 bytes spill stores, 328 bytes spill loads



215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Ensemble Accuracy: 0.7360605619449702
